In [1]:
from models.victim_params import VictimParams
from victim_circ_gen import random_2_qubit_circuit_one_way_cnot, random_2_qubit_circuit_two_way_cnot
import pickle
from itertools import permutations, combinations

from qiskit import QuantumCircuit, QuantumRegister

# One-way CNOT

In [ ]:
params = VictimParams(2, 50, list(range(20)), 60)
qcs = []

for no_of_cnots in params.cnot_vals:
    for number_of_cnots in range(params.no_of_circuits_per_cnot_val):
        qc = random_2_qubit_circuit_one_way_cnot(no_of_cnots, params.max_circuit_depth)
        qcs.append(qc)

pickle.dump( (qcs, params), open( "pickles/victim_circs/one_way_cnot_{}.p".format(str(params.uuid)), "wb" ) )

# Two-way CNOTs

In [ ]:
params = VictimParams(2, 50, list(range(20)), 60)
qcs = []

for no_of_cnots in params.cnot_vals:
    for number_of_cnots in range(params.no_of_circuits_per_cnot_val):
        qc = random_2_qubit_circuit_two_way_cnot(no_of_cnots, params.max_circuit_depth)
        qcs.append(qc)

pickle.dump( (qcs, params), open( "pickles/victim_circs/two_way_cnot_{}.p".format(str(params.uuid)), "wb" ) )

# Bigger Victim - 4 qubits chain with depth 1

In [2]:
victim_qregs = QuantumRegister(4, name='vic')
a, b, c, d = victim_qregs

In [3]:
neighbours = {
    a: (b, ),
    b: (a, c),
    c: (b, d),
    d: (c, )
}

str_qubits = { a: 'a', b: 'b', c: 'c', d: 'd' }

In [4]:
links = set()
for ctrl, target in permutations(neighbours.keys(), 2):
    if target in neighbours[ctrl]:
        links.add((ctrl, target))

In [6]:
qcs = []
for number_of_cnots in (1, 2):
    for cnot_ops in combinations(links, number_of_cnots):
        qc = QuantumCircuit(victim_qregs)
        names = []
        for ctrl_q, target_q in cnot_ops:
            qc.cx(ctrl_q, target_q)
            names.append((str_qubits[ctrl_q], str_qubits[target_q]))
        names.sort()
        qc.name = str(names)
        depth = qc.depth()
        if depth == 1:
            qcs.append(qc)

In [7]:
len(qcs)

10

In [8]:
pickle.dump( qcs, open( "pickles/victim_circs/four_qubit_chain_d1.p", "wb" ) )

# Bigger Victim - 4 qubits chain with depth 0, 1, 3, 5

In [2]:
victim_qregs = QuantumRegister(4, name='vic')
a, b, c, d = victim_qregs
qcs_per_reps = {}

## Zero depth

In [3]:
qc0 = QuantumCircuit(victim_qregs)
qcs_per_reps[0] = [qc0]

## Non-zero depth

In [4]:
non_zero_reps = (1, 3, 5)

In [5]:
cx_gates_applied_per_circ = [
    [[a, b]],
    [[b, a]], 
    [[b, c]], 
    [[c, b]], 
    [[c, d]], 
    [[d, c]], 
    [[a, b], [c, d]], 
    [[a, b], [d, c]],
    [[b, a], [c, d]], 
    [[b, a], [d, c]]
]

In [6]:
for rep in non_zero_reps:
    qcs = []
    for cx_gates_applied in cx_gates_applied_per_circ:
        qc = QuantumCircuit(victim_qregs)
        for i in range(rep):
            for ctrl, target in cx_gates_applied:
                qc.cx(ctrl, target)
        qcs.append(qc)
    qcs_per_reps[rep] = qcs

## Save to pickle

In [7]:
pickle.dump( qcs_per_reps, open( "pickles/victim_circs/four_qubit_chain_0135.p", "wb" ) )